In [1]:
import psycopg2
import sqlalchemy as db
import numpy as np
import pandas as pd
from sqlalchemy import text
from connector import connect_to
import plotly.express as px

In [2]:
with connect_to('postgres') as conn:
    df=pd.read_sql("""
	select 
		sum(i.total)
		, count(*)
		, billing_country as country
		, case 
			when billing_country in ('Spain', 'Italy', 'Hungary', 'Czech Republic', 'Belgium', 'Sweden', 'Noraway', 'France', 'United Kingdom', 'Netherlands', 'Austria', 'Poland', 'Ireland', 'Germany','Denmark', 'Finland', 'Portugal') then 'Europe'
			when billing_country in ('India', 'Australia') then 'Asia'
			when billing_country in ('USA', 'Canada') then 'North America'
			when billing_country in ('Argentina', 'Chile', 'Brazil') then 'South America'
		end as region
		, round(sum(i.total)/count(*),2) as avg_check
	from invoice i 
	group by billing_country, region
""", conn)

In [3]:
fig = px.scatter(
        data_frame=df,
        x='sum',
        y='count',
        color='region',
        color_discrete_map={'Asia':'yellow','Europe':'royalblue', 'North America':'red'},
        hover_name='country',
        hover_data={'sum':True, 'count':True, 'avg_check':True}
    )
fig.update_layout(
    {'title':{'text':'Sum vs Count', 'font':{'size':30, 'color':'gray'}}, 
     'xaxis': {'tickfont': {'size': 14, 'color': 'gray'}},
     'yaxis': {'tickfont': {'size': 14, 'color': 'gray'}}}
)
fig.show()

In [4]:
with connect_to('postgres') as conn:
    df2=pd.read_sql("""
    select 
        count(*)
        , t.genre_id
        , g.name
    from track t
    left join genre g on t.genre_id = g.genre_id
    group by t.genre_id, g.name
    order by t.genre_id
""", conn)

In [5]:
a={}
for i in df2['name']:
    if i in ['Jazz', 'Rock', 'Alternative & Punk'] :
        a[i] = 'black'
    else: a[i] = 'gray'
fig=px.bar(
    data_frame = df2,
    x='name',
    y='count',
    color='name',
    color_discrete_map=a    
)
fig.update_layout({
    'showlegend':False,
    'title':{'text':'<i>Name && Count</i>'},
    'template':'simple_white'
})